In [18]:
import pandas as pd
import re
import trafilatura

# URL 설정
url = "https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A005930&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701"

# Trafilatura로 HTML 본문 추출
downloaded = trafilatura.fetch_url(url)

if downloaded:
    text = trafilatura.extract(downloaded, include_tables=True)
else:
    print("URL에서 데이터를 가져올 수 없습니다.")
    text = ""

# 매출액, 당기순이익, 영업활동으로인한현금흐름, 투자활동으로인한현금흐름, 재무활동으로인한현금흐름 데이터 추출
pattern = r"(매출액|당기순이익|영업활동으로인한현금흐름|투자활동으로인한현금흐름|재무활동으로인한현금흐름)\s*\|\s*(.*?)\n"
matches = re.findall(pattern, text)

# 데이터 정리
data = []
columns = ["항목", "2021/12", "2022/12", "2023/12", "2024/06", "전년동기", "전년동기(%)"]

# 항목 이름 변경: 영업활동으로인한현금흐름 -> 영업활동현금흐름, 투자활동으로인한현금흐름 -> 투자활동현금흐름, 재무활동으로인한현금흐름 -> 재무활동현금흐름
matches = [(항목명.replace("영업활동으로인한현금흐름", "영업활동현금흐름")
            .replace("투자활동으로인한현금흐름", "투자활동현금흐름")
            .replace("재무활동으로인한현금흐름", "재무활동현금흐름"), 값) 
           for 항목명, 값 in matches]

# 중복 항목 처리
처리된_항목 = set()  # 이미 처리된 항목 저장

for match in matches:
    항목명 = match[0]
    
    # 이미 처리된 항목은 건너뜀
    if 항목명 in 처리된_항목:
        continue

    # 항목 처리
    처리된_항목.add(항목명)  # 처리 완료된 항목 추가
    row = [항목명] + [x.strip() for x in match[1].split("|")]
    
    # 열 수 맞추기: 데이터가 부족하면 NaN으로 채움
    if len(row) < len(columns):
        row += [None] * (len(columns) - len(row))
    # 데이터 초과 시, 필요한 만큼만 사용
    elif len(row) > len(columns):
        row = row[:len(columns)]
    
    data.append(row)

# pandas DataFrame 생성
df = pd.DataFrame(data, columns=columns)

# 숫자로 변환할 수 있는 항목들만 추출 (쉼표 제거 후)
def clean_and_convert(value):
    if value is None:
        return None
    try:
        return float(value.replace(",", ""))
    except ValueError:
        return None

# 숫자 데이터로 변환
for col in df.columns[1:]:
    df[col] = df[col].apply(clean_and_convert)

# 영업활동, 투자활동, 재무활동의 현금흐름 합을 계산하여 새로운 "잉여현금흐름" 행 추가
영업활동_col = "영업활동현금흐름"
투자활동_col = "투자활동현금흐름"
재무활동_col = "재무활동현금흐름"

# "영업활동현금흐름", "투자활동현금흐름", "재무활동현금흐름"이 존재할 때만 처리
if 영업활동_col in df["항목"].values and 투자활동_col in df["항목"].values and 재무활동_col in df["항목"].values:
    영업활동 = df[df["항목"] == 영업활동_col].iloc[0, 1:]
    투자활동 = df[df["항목"] == 투자활동_col].iloc[0, 1:]
    재무활동 = df[df["항목"] == 재무활동_col].iloc[0, 1:]
    
    # 잉여현금흐름 계산 (영업활동 + 투자활동 + 재무활동)
    잉여현금흐름 = 영업활동 + 투자활동 + 재무활동

    # 잉여현금흐름을 새로운 행으로 추가
    new_row = ["잉여현금흐름"] + 잉여현금흐름.tolist()
    df.loc[len(df)] = new_row

# 결과 출력
print(df)


         항목    2021/12    2022/12    2023/12    2024/06       전년동기  전년동기(%)
0       매출액  2796048.0  3022314.0  2589355.0  2250826.0  1911556.0     17.7
1     당기순이익   399075.0   556541.0   154871.0   266970.0    91423.0    192.0
2  영업활동현금흐름   651054.0   621813.0   441374.0   509604.0        NaN      NaN
3  투자활동현금흐름  -330478.0  -316028.0  -169228.0  -735138.0        NaN      NaN
4  재무활동현금흐름  -239910.0  -193900.0   -85931.0   -50162.0        NaN      NaN
5    잉여현금흐름    80666.0   111885.0   186215.0  -275696.0        NaN      NaN


In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL 설정
url = "https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A005930&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701"

# 웹 페이지 요청
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36"
}
response = requests.get(url, headers=headers)

# 요청 성공 여부 확인
if response.status_code != 200:
    print(f"페이지를 가져오지 못했습니다. 상태 코드: {response.status_code}")
    exit()

# HTML 파싱
soup = BeautifulSoup(response.content, 'html.parser')

# 열 헤더 추출
table = soup.find("table", {"class": "us_table_ty1 h_fix zigbg_no"})
if table:
    columns = [th.get_text(strip=True) for th in table.find("thead").find_all("th")]
else:
    print("테이블을 찾을 수 없습니다.")
    exit()

# 부채비율 데이터 추출
row1 = soup.find("tr", {"id": "p_grid1_3"})  # "부채비율" 행 선택
if row1:
    item_name1 = row1.select_one('th .txt_acd span').get_text(strip=True)
    values1 = [td.get_text(strip=True) for td in row1.find_all("td", {"class": "r"})]
else:
    item_name1 = "부채비율"
    values1 = ["N/A"] * 5 # 데이터가 없을 경우 "N/A"로 채움
# 유보율 데이터 추출
row2 = soup.find("tr", {"id": "p_grid1_4"})  # "유보율" 행 선택
if row2:
    item_name2 = row2.select_one('th .txt_acd span').get_text(strip=True)
    values2 = [td.get_text(strip=True) for td in row2.find_all("td", {"class": "r"})]
else:
    item_name2 = "유보율"
    values2 = ["N/A"] * 5
row3 = soup.find("tr", {"id": "p_grid1_18"})  # ROE" 행 선택
if row3:
    item_name3 = row3.select_one('th .txt_acd span').get_text(strip=True)
    values3 = [td.get_text(strip=True) for td in row3.find_all("td", {"class": "r"})]
else:
    item_name3 = "ROE"
    values3 = ["N/A"] * 5
row4 = soup.find("tr", {"id": "p_grid1_20"})  # "총자산회전율" 행 선택
if row4:
    item_name4 = row4.select_one('th .txt_acd span').get_text(strip=True)
    values4 = [td.get_text(strip=True) for td in row4.find_all("td", {"class": "r"})]
else:
    item_name2 = "총자산회전율"
    values4 = ["N/A"] * 5

# 데이터프레임 생성
data = [
    [item_name1] + values1,
    [item_name2] + values2,
    [item_name3] + values3,
    [item_name4] + values4
]
df = pd.DataFrame(data, columns=columns)

# 결과 출력
df


,IFRS(연결),2020/12,2021/12,2022/12,2023/12,2024/09
0,부채비율,37.1,39.9,26.4,25.4,27.2
1,유보율,"29,723.5","32,906.5","38,360.3","39,256.9","41,810.5"
2,ROE,10.0,13.9,17.1,4.1,9.5
3,총자산회전율,0.7,0.7,0.7,0.6,0.6


In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL 설정
url = "https://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A005930&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701"

# 웹 페이지 요청
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36"
}
response = requests.get(url, headers=headers)

# 요청 성공 여부 확인
if response.status_code != 200:
    print(f"페이지를 가져오지 못했습니다. 상태 코드: {response.status_code}")
    exit()

# HTML 파싱
soup = BeautifulSoup(response.content, 'html.parser')

# 열 헤더 추출
table = soup.find("table", {"class": "us_table_ty1 h_fix zigbg_no"})
if table:
    columns = [th.get_text(strip=True) for th in table.find("thead").find_all("th")]
else:
    print("테이블을 찾을 수 없습니다.")
    exit()

# 데이터 추출 함수
def extract_row_data(row_id, default_name):
    row = soup.find("tr", {"id": row_id})
    if row:
        item_name = row.select_one('th .txt_acd span').get_text(strip=True) if row.select_one('th .txt_acd span') else default_name
        values = [td.get_text(strip=True) for td in row.find_all("td", {"class": "r"})]
    else:
        item_name = default_name
        values = ["N/A"] * 5  # 데이터가 없을 경우 "N/A"로 채움
    return item_name, values

# "시가총액" 데이터 추출
market_cap_row = None
for row in soup.find_all("tr"):
    th = row.find("th")
    if th and "시가총액" in th.get_text(strip=True):
        market_cap_row = row
        break

if market_cap_row:
    market_cap_values = [td.get_text(strip=True) for td in market_cap_row.find_all("td", {"class": "r"})]
    market_cap_name = "시가총액"
else:
    market_cap_values = ["N/A"] * 5
    market_cap_name = "시가총액"

# (홀수 번째 데이터)
market_cap = market_cap_values[::2] if market_cap_values else ["N/A"] * 5

# 다른 데이터 추출
item_name5, values5 = extract_row_data("p_grid1_9", "PER")
item_name6, values6 = extract_row_data("p_grid1_12", "PBR")
item_name7, values7 = extract_row_data("p_grid1_14", "EV/EBITDA")

# 데이터프레임 생성
data = [
    [market_cap_name] + market_cap,
    [item_name5] + values5,
    [item_name6] + values6,
    [item_name7] + values7
]
df = pd.DataFrame(data, columns=columns)

# 결과 출력
print(df)


     IFRS 연결    2020/12    2021/12    2022/12    2023/12    2024/09
0       시가총액  5,441,168  6,099,040  5,305,928  5,198,938  5,814,198
1        PER      21.09      13.55       6.86      36.84           
2        PBR       2.06       1.80       1.09       1.51       1.11
3  EV/EBITDA       8.33       6.23       4.62      11.72           


In [ ]:
import pandas as pd
import re
import trafilatura
import requests
from bs4 import BeautifulSoup
import json  # JSON 파일을 읽기 위한 모듈

# JSON 파일에서 기업명과 고유 코드를 로드하는 함수
def load_company_codes(file_path="extracted_dict.json"):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            company_codes = json.load(file)
        return company_codes
    except FileNotFoundError:
        print(f"{file_path} 파일을 찾을 수 없습니다.")
        return {}
    except json.JSONDecodeError:
        print(f"{file_path} 파일을 읽을 수 없습니다. 파일 형식이 올바른지 확인하세요.")
        return {}

# URL 생성 함수
def generate_urls(company_code):
    url_finance = f"https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode={company_code}&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701"
    url_ratio = f"https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode={company_code}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701"
    url_invest = f"https://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode={company_code}&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701"
    
    return url_finance, url_ratio, url_invest

# Trafilatura로 HTML 본문 추출
def fetch_data(url):
    downloaded = trafilatura.fetch_url(url)
    if downloaded:
        return trafilatura.extract(downloaded, include_tables=True)
    else:
        print(f"URL에서 데이터를 가져올 수 없습니다: {url}")
        return ""

# 매출액, 당기순이익, 영업활동으로인한현금흐름 등 데이터 추출

def extract_finance_data(text):
    # 정규식 패턴: 매출액, 당기순이익, 영업활동현금흐름, 투자활동현금흐름, 재무활동현금흐름
    pattern = r"(매출액|당기순이익|영업활동으로인한현금흐름|투자활동으로인한현금흐름|재무활동으로인한현금흐름)\s*\|\s*(.*?)\n"
    matches = re.findall(pattern, text)

    # 항목 처리
    처리된_항목 = set()
    data = []
    columns = ["항목", "2021/12", "2022/12", "2023/12", "2024/09"]

    for match in matches:
        항목명 = match[0].replace("영업활동으로인한현금흐름", "영업활동현금흐름") \
                         .replace("투자활동으로인한현금흐름", "투자활동현금흐름") \
                         .replace("재무활동으로인한현금흐름", "재무활동현금흐름")

        if 항목명 in 처리된_항목:
            continue
        
        처리된_항목.add(항목명)
        row = [항목명] + [x.strip() for x in match[1].split("|")]
        
        if len(row) < len(columns):
            row += [None] * (len(columns) - len(row))
        elif len(row) > len(columns):
            row = row[:len(columns)]
        
        data.append(row)

    df = pd.DataFrame(data, columns=columns)
    
    # 잉여현금흐름(Final Cash Flow) 계산
    if {"영업활동현금흐름", "투자활동현금흐름", "재무활동현금흐름"}.issubset(df["항목"].values):
        # 쉼표를 제거하고 float으로 변환
        영업활동현금흐름 = df.loc[df["항목"] == "영업활동현금흐름"].iloc[:, 1:].replace({',': ''}, regex=True).values.astype(float)
        투자활동현금흐름 = df.loc[df["항목"] == "투자활동현금흐름"].iloc[:, 1:].replace({',': ''}, regex=True).values.astype(float)
        재무활동현금흐름 = df.loc[df["항목"] == "재무활동현금흐름"].iloc[:, 1:].replace({',': ''}, regex=True).values.astype(float)

        # 합산하여 잉여현금흐름 계산
        fcf = 영업활동현금흐름 + 투자활동현금흐름 + 재무활동현금흐름
        fcf_row = ["잉여현금흐름"] + fcf[0].tolist()

        # 잉여현금흐름 추가
        fcf_df = pd.DataFrame([fcf_row], columns=columns)
        df = pd.concat([df, fcf_df], ignore_index=True)

    # 매출액, 당기순이익, 영업활동현금흐름, 잉여현금흐름만 포함
    df = df[df["항목"].isin(["매출액", "당기순이익", "영업활동현금흐름", "잉여현금흐름"])]
    
    return df


def extract_row_data(soup, row_id, default_name):
    row = soup.find("tr", {"id": row_id})
    if row:
        item_name = row.select_one('th .txt_acd span').get_text(strip=True) if row.select_one('th .txt_acd span') else default_name
        values = [td.get_text(strip=True) for td in row.find_all("td", {"class": "r"})]
    else:
        item_name = default_name
        values = ["N/A"] * 5
    return item_name, values

def extract_ratio_data(url):
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    if response.status_code != 200:
        print(f"페이지를 가져오지 못했습니다. 상태 코드: {response.status_code}")
        return pd.DataFrame()

    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find("table", {"class": "us_table_ty1 h_fix zigbg_no"})
    if table:
        columns = [th.get_text(strip=True) for th in table.find("thead").find_all("th")]
    else:
        print("테이블을 찾을 수 없습니다.")
        return pd.DataFrame()

    item_name1, values1 = extract_row_data(soup, "p_grid1_3", "부채비율")
    item_name2, values2 = extract_row_data(soup, "p_grid1_4", "유보율")
    item_name3, values3 = extract_row_data(soup, "p_grid1_18", "ROE")
    item_name4, values4 = extract_row_data(soup, "p_grid1_20", "총자산회전율")

    data = [
        [item_name1] + values1,
        [item_name2] + values2,
        [item_name3] + values3,
        [item_name4] + values4
    ]
    
    df = pd.DataFrame(data, columns=columns)
    df.rename(columns={df.columns[0]: "항목"}, inplace=True)
    
    return df

def extract_invest_data(url):
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    if response.status_code != 200:
        print(f"페이지를 가져오지 못했습니다. 상태 코드: {response.status_code}")
        return pd.DataFrame()

    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find("table", {"class": "us_table_ty1 h_fix zigbg_no"})
    if table:
        columns = [th.get_text(strip=True) for th in table.find("thead").find_all("th")]
    else:
        print("테이블을 찾을 수 없습니다.")
        return pd.DataFrame()

    market_cap_row = None
    for row in soup.find_all("tr"):
        th = row.find("th")
        if th and "시가총액" in th.get_text(strip=True):
            market_cap_row = row
            break

    if market_cap_row:
        market_cap_values = [td.get_text(strip=True) for td in market_cap_row.find_all("td", {"class": "r"})]
        market_cap_name = "시가총액"
    else:
        market_cap_values = ["N/A"] * 5
        market_cap_name = "시가총액"

    market_cap = market_cap_values[::2] if market_cap_values else ["N/A"] * 5

    item_name5, values5 = extract_row_data(soup, "p_grid1_9", "PER")
    item_name6, values6 = extract_row_data(soup, "p_grid1_12", "PBR")
    item_name7, values7 = extract_row_data(soup, "p_grid1_14", "EV/EBITDA")

    data = [
        [market_cap_name] + market_cap,
        [item_name5] + values5,
        [item_name6] + values6,
        [item_name7] + values7
    ]
    
    df = pd.DataFrame(data, columns=columns)
    df.rename(columns={df.columns[0]: "항목"}, inplace=True)
    
    return df

def merge_dataframes(df_finance, df_ratio, df_invest, company_name):
    df_finance["기업명"] = company_name  # 기업명 열 추가
    df_ratio["기업명"] = company_name
    df_invest["기업명"] = company_name

    combined_df = pd.concat([df_finance, df_ratio, df_invest], ignore_index=True, sort=False)
    combined_df = combined_df.fillna("N/A")
    columns = ['기업명', '항목'] + sorted([col for col in combined_df.columns if col not in ['기업명', '항목']])
    combined_df = combined_df[columns]
    
    return combined_df

# 모든 기업에 대해 데이터 처리 및 저장
def process_and_save_all_data():
    company_codes = load_company_codes("extracted_dict.json")
    
    all_data = []  # 모든 데이터를 저장할 리스트

    for company_name, company_code in company_codes.items():
        print(f"데이터 처리 중: {company_name} ({company_code})")
        
        # URL 생성
        url_finance, url_ratio, url_invest = generate_urls(company_code)
        
        # Trafilatura로 금융 데이터 가져오기
        finance_data = fetch_data(url_finance)
        df_finance = extract_finance_data(finance_data)
        
        # 비율 데이터와 투자 데이터를 가져오기
        df_ratio = extract_ratio_data(url_ratio)
        df_invest = extract_invest_data(url_invest)

        # DataFrame들을 병합
        combined_df = merge_dataframes(df_finance, df_ratio, df_invest, company_name)
        
        # 기업명별로 데이터를 추가
        all_data.append(combined_df)

    # 모든 기업의 데이터를 하나의 DataFrame으로 합침
    final_df = pd.concat(all_data, ignore_index=True)

    # 하나의 CSV로 저장
    final_df.to_csv("all_company_data.csv", index=False, encoding="utf-8-sig")

# 실행
process_and_save_all_data()


데이터 처리 중: 삼성전자 (A005930)
데이터 처리 중: NAVER (A035420)
데이터 처리 중: 삼성SDI (A006400)
데이터 처리 중: 크래프톤 (A259960)
데이터 처리 중: 삼성에스디에스 (A018260)
데이터 처리 중: LG이노텍 (A011070)
데이터 처리 중: 한미반도체 (A042700)
데이터 처리 중: DB하이텍 (A000990)
데이터 처리 중: 솔루스첨단소재 (A336370)
데이터 처리 중: 넷마블 (A251270)
데이터 처리 중: 삼성전기 (A009150)
데이터 처리 중: 에코프로 (A086520)
데이터 처리 중: 엔씨소프트 (A036570)
데이터 처리 중: 솔브레인 (A357780)
데이터 처리 중: 현대오토에버 (A307950)
데이터 처리 중: 동진쎄미켐 (A005290)
데이터 처리 중: 리노공업 (A058470)
데이터 처리 중: 카카오 (A035720)
데이터 처리 중: LX세미콘 (A108320)
데이터 처리 중: 해성디에스 (A195870)
데이터 처리 중: 포스코DX (A022100)
데이터 처리 중: HPSP (A403870)
데이터 처리 중: SOOP (A067160)
데이터 처리 중: 피엔티 (A137400)
데이터 처리 중: 티씨케이 (A064760)
데이터 처리 중: 더블유게임즈 (A192080)
데이터 처리 중: 이오테크닉스 (A039030)
데이터 처리 중: 제이앤티씨 (A204270)
데이터 처리 중: 더블유씨피 (A393890)
데이터 처리 중: 웹젠 (A069080)
데이터 처리 중: 월덱스 (A101160)
데이터 처리 중: KG이니시스 (A035600)
데이터 처리 중: 펄어비스 (A263750)
데이터 처리 중: 엘오티베큠 (A083310)
데이터 처리 중: 롯데이노베이트 (A286940)
데이터 처리 중: 두산테스나 (A131970)
데이터 처리 중: 쏠리드 (A050890)
데이터 처리 중: 컴투스 (A078340)
데이터 처리 중: 피에스케이 (A319660)
데

In [77]:
import json

# JSON 파일에서 딕셔너리 로드
with open("extracted_dict.json", "r", encoding="utf-8") as f:
    extracted_dict = json.load(f)

# 검색할 키값을 입력받음
search_key = input("검색할 키값을 입력하세요: ")

# 딕셔너리에서 키값을 검색하고 값 출력
if search_key in extracted_dict:
    print(f"'{search_key}'의 값은: {extracted_dict[search_key]}")
else:
    print(f"'{search_key}'에 해당하는 값이 없습니다.")


'삼성전자'의 값은: A005930


In [ ]:
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
import time
import json

# Chrome WebDriver 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # 브라우저 창을 열지 않음
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# WebDriver 실행 (webdriver_manager로 자동 설치)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# URL로 이동
url = "https://comp.fnguide.com/SVO2/asp/SVD_UJRank.asp?pGB=1&gicode=A000270&cID=&MenuYn=Y&ReportGB=&NewMenuID=301&stkGb=701"
driver.get(url)

# 페이지 로드 대기
time.sleep(3)

# selUpjong 드롭다운에서 IT 업종 선택
try:
    # ID가 'selUpjong'인 드롭다운 요소를 대기
    dropdown = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'selUpjong'))
    )
    select_element = Select(dropdown)  # Select 객체로 변환
    select_element.select_by_value("FI00.45")  # IT 업종의 value 값 선택

    time.sleep(2)  # 변경 내용 반영 대기

    # 조회 버튼 클릭
    search_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'btnSearch'))
    )
    search_button.click()  # 조회 버튼 클릭

    time.sleep(5)  # 페이지 로드 대기
except Exception as e:
    print("업종 선택 또는 조회 버튼 클릭에 실패했습니다:", e)
    driver.quit()
    exit()

# class="l tbold"인 <td> 태그 찾기
td_elements = driver.find_elements(By.CLASS_NAME, "l.tbold")

# 딕셔너리 형태로 값들을 저장할 변수
extracted_dict = {}

# 각 <td> 태그 내에서 하이퍼링크 (a 태그) 추출
for td in td_elements:
    # <td> 안의 모든 <a> 태그 찾기
    a_tags = td.find_elements(By.TAG_NAME, 'a')
    
    # 각 <a> 태그의 href 속성(링크) 추출
    for a in a_tags:
        link = a.get_attribute('href')
        
        # 'javascript:ViewReport(...)'에서 인자값만 추출하는 정규 표현식
        match = re.search(r"javascript:ViewReport\('([^']+)'\)", link)
        
        if match:
            # a 태그의 텍스트를 키로, 추출한 인자값을 값으로 딕셔너리에 추가
            extracted_dict[a.text] = match.group(1)
            
            # 최대 100개만 추출
            if len(extracted_dict) >= 100:
                break

    # 100개 이상 추출되면 반복 종료
    if len(extracted_dict) >= 100:
        break

# 추출한 딕셔너리를 JSON 파일로 저장
with open("extracted_dict.json", "w", encoding="utf-8") as f:
    json.dump(extracted_dict, f, ensure_ascii=False, indent=4)

# WebDriver 종료
driver.quit()
'all_company_data.csv'

selUpjong 드롭다운에서 IT 업종이 선택되었습니다.
조회 버튼이 클릭되었습니다.


In [ ]:
import pandas as pd

# CSV 파일 읽기 (파일 경로를 넣어야 함)
file_path = 'all_company_data.csv'
df = pd.read_csv(file_path)

# NaN을 제외하고 평균 계산 (숫자형 데이터만 사용)
df = df.replace({'N/A': None, ',': ''}, regex=True)  # 'N/A'를 None으로, ',' 제거
df.iloc[:, 2:] = df.iloc[:, 2:].apply(pd.to_numeric, errors='coerce')  # 숫자형으로 변환

# '기업명' 열을 제외하고, 각 항목별로 기업들의 평균 계산
averages = df.drop(columns='기업명').groupby('항목').mean()

# 평균값을 소수점 1자리까지 반올림 후 문자열로 변환
averages = averages.round(1).astype(str)

# 평균값 DataFrame 정리 (기업명: average 추가)
averages = averages.reset_index()  # 항목을 인덱스에서 컬럼으로 이동
averages['기업명'] = 'average'  # 기업명 추가

# 원하는 항목 순서대로 리스트 생성
desired_order = [
    '매출액', '당기순이익', '영업활동현금흐름', '잉여현금흐름', '부채비율', 
    '유보율', 'ROE', '총자산회전율', '시가총액', 'PER', 'PBR', 'EV/EBITDA'
]

# 항목 순서를 원하는 대로 재정렬
averages = averages.set_index('항목').reindex([item for item in desired_order]).reset_index()

# 원본 데이터에 평균값 추가
df = pd.concat([df, averages], ignore_index=True)

# 모든 값을 문자열로 변환 (소수점 형식 유지)
df = df.astype(str)

# CSV 파일로 저장 (덮어쓰기)
df.to_csv(file_path, index=False)


평균값을 '기업명=average'로 추가한 데이터를 all_company_data.csv에 저장했습니다.


In [9]:
import pandas as pd

# CSV 파일 읽기
file_path = "all_company_data.csv"
data = pd.read_csv(file_path)

# "삼성전자" 데이터만 필터링
samsung_data = data[data["기업명"] == "삼성SDI"]

# 결과 확인
print(samsung_data)


      기업명         항목  2020/12  2021/12  2022/12   2023/12 2024/09  2024/06
24  삼성SDI        매출액      NaN  135,532  201,241   227,083  95,810      NaN
25  삼성SDI      당기순이익      NaN   12,504   20,394    20,660   5,878      NaN
26  삼성SDI   영업활동현금흐름      NaN   21,760   26,411    21,035  -6,246      NaN
27  삼성SDI     잉여현금흐름      NaN   8092.0   3236.0  -10986.0  1168.0      NaN
28  삼성SDI       부채비율     61.2     70.0     75.7      71.0     NaN     78.3
29  삼성SDI        유보율  3,634.9  4,118.9  4,618.3   5,186.2     NaN  5,451.2
30  삼성SDI        ROE      4.5      8.5     12.5      11.5     NaN      6.3
31  삼성SDI     총자산회전율      0.6      0.6      0.7       0.7     NaN      0.5
32  삼성SDI       시가총액  436,776  569,216  528,265   550,982     NaN  338,612
33  삼성SDI        PER    76.91    39.41    21.31     16.53     NaN      NaN
34  삼성SDI        PBR     3.32     3.06     2.47      1.76     NaN     1.26
35  삼성SDI  EV/EBITDA    26.38    20.80    13.41     11.28     NaN      NaN


In [46]:
import pandas as pd

# CSV 파일 읽기
file_path = "all_company_data.csv"
data = pd.read_csv(file_path)

# NaN 값을 채우기 위한 함수
def get_filled_latest_value(row):
    # NaN 값을 가장 최근 데이터로 채움
    for column in reversed(row.index[2:]):  # "2020/12" 이후 컬럼만 탐색
        if pd.notna(row[column]):
            return row[column]
    return None  # 모든 값이 NaN일 경우

# 기업별로 데이터 변형
result_dfs = []  # 각 기업별 결과를 저장할 리스트
for company in data["기업명"].unique():
    # 해당 기업 데이터 필터링
    company_data = data[data["기업명"] == company]
    
    # NaN 없이 최근 값 추출
    filled_values = company_data.apply(get_filled_latest_value, axis=1)
    
    # 새로운 데이터프레임 생성
    result_df = pd.DataFrame({
        "기업명": company_data["기업명"],
        "항목": company_data["항목"],
        "최근값": filled_values
    })
    
    result_dfs.append(result_df)

# 모든 기업 데이터 합치기
final_result = pd.concat(result_dfs, ignore_index=True)

# 결과 확인
print(final_result)

# 결과를 CSV로 저장
final_result.to_csv("processed_company_data.csv", index=False, encoding="utf-8-sig")


          기업명         항목        최근값
0        삼성전자        매출액  2250826.0
1        삼성전자      당기순이익   266970.0
2        삼성전자   영업활동현금흐름   509604.0
3        삼성전자     잉여현금흐름  -275696.0
4        삼성전자       부채비율       27.2
...       ...        ...        ...
1207  average     총자산회전율        0.7
1208  average       시가총액    30089.7
1209  average        PER      109.7
1210  average        PBR        3.4
1211  average  EV/EBITDA       32.1

[1212 rows x 3 columns]


In [72]:
import pandas as pd
import numpy as np

# CSV 파일 읽기
file_path = "processed_company_data.csv"
data = pd.read_csv(file_path)

# 점수 계산 함수 (1 ~ 10 점수)
def assign_scores(df, column="최근값"):
    scores = list(range(1, 11))  # 점수는 1부터 10까지
    
    # NaN 값 제외하고 계산
    valid_values = df[column].dropna()
    if valid_values.empty:
        df["점수"] = np.nan
        return df
    
    # 각 값의 순위를 계산하여 점수 부여
    rank = valid_values.rank(pct=True)  # 값들을 0~1 사이로 정규화된 순위로 변환
    df["점수"] = pd.cut(rank, bins=np.linspace(0, 1, 11), labels=scores, include_lowest=True).astype(int)
    
    return df

# 항목별로 점수 계산
scored_data = []
for 항목 in data["항목"].unique():
    항목_data = data[data["항목"] == 항목].copy()
    항목_data = assign_scores(항목_data)

    # 반대로 계산할 항목 (예: 부채비율, PER, PBR, EV/EBITDA)
    if 항목 in ["부채비율", "PER", "PBR", "EV/EBITDA"]:
        항목_data["점수"] = 11 - 항목_data["점수"]  # 계산된 점수에서 11을 빼서 반전
    
    scored_data.append(항목_data)

# 결과 합치기
final_data = pd.concat(scored_data, ignore_index=True)

# 기업명 기준 정렬
final_data = final_data.sort_values(by=["기업명", "항목"], ignore_index=True)

# NaN 점수 기본 처리 (0점 할당)
final_data["점수"] = final_data["점수"].fillna(0).astype(int)

# 결과 확인
print(final_data)

# 결과를 CSV로 저장
final_data.to_csv("scored_company_data_sorted.csv", index=False, encoding="utf-8-sig")


         기업명         항목       최근값  점수
0      DB하이텍  EV/EBITDA      6.52   9
1      DB하이텍        PBR      1.12   8
2      DB하이텍        PER      9.87  10
3      DB하이텍        ROE     12.10   7
4      DB하이텍      당기순이익   1080.00   9
...      ...        ...       ...  ..
1207  현대오토에버       시가총액  54683.00   9
1208  현대오토에버   영업활동현금흐름   2383.00  10
1209  현대오토에버        유보율  12045.50   9
1210  현대오토에버     잉여현금흐름  -1895.00   1
1211  현대오토에버     총자산회전율      1.20   9

[1212 rows x 4 columns]


In [74]:
final_data

,기업명,항목,최근값,점수
0,DB하이텍,EV/EBITDA,6.52,9
1,DB하이텍,PBR,1.12,8
2,DB하이텍,PER,9.87,10
3,DB하이텍,ROE,12.10,7
4,DB하이텍,당기순이익,1080.00,9
...,...,...,...,...
1207,현대오토에버,시가총액,54683.00,9
1208,현대오토에버,영업활동현금흐름,2383.00,10
1209,현대오토에버,유보율,12045.50,9
1210,현대오토에버,잉여현금흐름,-1895.00,1


In [36]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# CSV 파일 읽기
file_path = "scored_company_data_sorted.csv"
data = pd.read_csv(file_path)

def plot_radar_chart(company_name):
    # 입력받은 기업명 데이터 필터링
    company_data = data[data['기업명'] == company_name]
    
    if company_data.empty:
        print(f"기업명 '{company_name}'에 해당하는 데이터가 없습니다.")
        return
    
    # 항목과 점수 및 최근값 가져오기
    categories = company_data['항목'].tolist()
    values = company_data['점수'].tolist()
    recent_values = company_data['최근값'].tolist()  # 최근값 추가
    
    # 방사형 차트를 위한 값 준비
    num_vars = len(categories)
    
    # 각도 계산
    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
    values += values[:1]  # 차트를 닫기 위해 첫 번째 값을 추가
    recent_values += recent_values[:1]  # 최근값도 동일하게 추가
    angles += angles[:1]  # 각도도 동일하게 닫기
    
    # Plotly를 사용하여 방사형 차트 생성 (면적 없이 선만 표시)
    fig = go.Figure(data=go.Scatterpolar(
        r=values,
        theta=categories,
        mode='markers+lines',  # 면적 없이 선만 그리기 위해 'lines' 사용
        marker=dict(size=8, color='blue'),
        text=[f'{categories[i]}: 최근값 = {recent_values[i]}' for i in range(num_vars)],
        hoverinfo='text'
    ))

    # 레이아웃 설정
    fig.update_layout(
        title=f"{company_name} 방사형 차트",
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 10]  # 최대 점수 10으로 설정
            ),
        ),
        showlegend=False
    )
    
    # 그래프 표시
    fig.show()

# 예제 실행
input_company = input("방사형 차트를 보고 싶은 기업명을 입력하세요: ")
plot_radar_chart(input_company)


In [79]:
import pandas as pd
import plotly.graph_objects as go

# CSV 파일 읽기
file_path = "scored_company_data_sorted.csv"
data = pd.read_csv(file_path)

# 점수 컬럼이 없으면 처리
if "점수" not in data.columns:
    print("점수 컬럼이 데이터에 없습니다. 점수를 먼저 계산해주세요.")
else:
    # 제일 좋은 기업 (점수 합계가 가장 높은 기업 1개)
    best_company = data.groupby('기업명')['점수'].sum().idxmax()

    # "average" 기업 데이터 필터링
    average_data = data[data['기업명'] == "average"]

    # 색상 리스트 (RGBA 값을 사용하여 투명도 설정)
    colors = [
        'rgba(0, 0, 255, 0.4)',  # 제일 좋은 기업 파란색
        'rgba(255, 0, 0, 0.4)'   # "average" 기업 빨간색
    ]

    # 방사형 차트 그리기
    def plot_radar_chart():
        # Plotly를 사용하여 빈 차트 생성
        fig = go.Figure()

        # 제일 좋은 기업에 대해 그래프 추가
        best_company_data = data[data['기업명'] == best_company]
        if not best_company_data.empty:
            categories = best_company_data['항목'].tolist()
            values = best_company_data['점수'].tolist()
            recent_values = best_company_data['최근값'].tolist()

            # 시작점과 끝점을 이어주기 위해 첫 번째 값을 마지막에 추가
            categories += categories[:1]
            values += values[:1]
            recent_values += recent_values[:1]

            # 호버 텍스트 생성
            hover_text = [f"{categories[i]}: {recent_values[i]}" for i in range(len(categories))]

            # 제일 좋은 기업 차트 추가
            fig.add_trace(go.Scatterpolar(
                r=values,
                theta=categories,
                mode='lines+markers',  # 선과 점 표시
                line=dict(color=colors[0], width=2),  # 선 색상과 두께 설정
                marker=dict(size=6),  # 점 크기 설정
                text=hover_text,  # 호버 텍스트 설정
                hoverinfo="text",  # 텍스트를 호버 정보로 표시
                name=best_company,
            ))

        # "average" 기업에 대해 그래프 추가
        if not average_data.empty:
            categories = average_data['항목'].tolist()
            values = average_data['점수'].tolist()
            recent_values = average_data['최근값'].tolist()

            # 시작점과 끝점을 이어주기 위해 첫 번째 값을 마지막에 추가
            categories += categories[:1]
            values += values[:1]
            recent_values += recent_values[:1]

            # 호버 텍스트 생성
            hover_text = [f"{categories[i]}: {recent_values[i]}" for i in range(len(categories))]

            # "average" 기업 차트 추가
            fig.add_trace(go.Scatterpolar(
                r=values,
                theta=categories,
                mode='lines+markers',  # 선과 점 표시
                line=dict(color=colors[1], width=2),  # 선 색상과 두께 설정
                marker=dict(size=6),  # 점 크기 설정
                text=hover_text,  # 호버 텍스트 설정
                hoverinfo="text",  # 텍스트를 호버 정보로 표시
                name="average",
            ))

        # 레이아웃 설정
        fig.update_layout(
            title="제일 좋은 기업과 Average 기업 방사형 차트",
            polar=dict(
                radialaxis=dict(
                    visible=True,
                    range=[0, 10]  # 최대 점수 10으로 설정
                ),
            ),
            showlegend=True,  # 범례 표시
        )

        # 그래프 표시
        fig.show()

    # 방사형 차트 그리기
    plot_radar_chart()
